## Initial settings

In [2]:
%%capture
import sys
!{sys.executable} -m pip install smdebug torch torchvision tqdm

In [72]:
import json
import boto3
import sagemaker
from sagemaker.pytorch import PyTorch
from sagemaker import get_execution_role
from sagemaker.estimator import Estimator
import matplotlib.pyplot as plt
from PIL import Image
import boto3
from time import sleep
import os

INPUT_LOCAL_PATH = './data/input'
INPUT_CONTENT_PATH = f'{INPUT_LOCAL_PATH}/content-images'
INPUT_STYLE_PATH = f'{INPUT_LOCAL_PATH}/style-images'

OUTPUT_LOCAL_PATH = './data/output'
OUTPUT_DOWNLOADS_PATH = f'{OUTPUT_LOCAL_PATH}/model-downloads'
OUTPUT_RESULTS_PATH = f'{OUTPUT_LOCAL_PATH}/model-results'

METADATA_LOCAL_PATH = './estimators_metadata'

In [17]:
session = sagemaker.Session()

bucket_sagemaker = session.default_bucket()
print("Default Bucket: {}".format(bucket_sagemaker))

region = session.boto_region_name
print("AWS Region: {}".format(region))

role = sagemaker.get_execution_role()
print("RoleArn: {}".format(role))

Default Bucket: sagemaker-us-east-1-798714931926
AWS Region: us-east-1
RoleArn: arn:aws:iam::798714931926:role/service-role/AmazonSageMaker-ExecutionRole-20230319T062026


## Copy content and style images to S3

In [7]:
CONTENT_IMAGES_BUCKET = 's3://sagemaker-studio-sr1encmpcxc/content-images/'
STYLE_IMAGES_BUCKET = 's3://sagemaker-studio-sr1encmpcxc/style-images/'
OUTPUT_BUCKET = 's3://sagemaker-studio-sr1encmpcxc/output/'

In [11]:
!aws s3 cp  ./data/input/content-images  {CONTENT_IMAGES_BUCKET}  --recursive

upload: data/input/content-images/sherlock.jpeg to s3://sagemaker-studio-sr1encmpcxc/content-images/sherlock.jpeg
upload: data/input/content-images/YellowLabradorLooking_new.jpeg to s3://sagemaker-studio-sr1encmpcxc/content-images/YellowLabradorLooking_new.jpeg
upload: data/input/content-images/jennifer-lawrence.png to s3://sagemaker-studio-sr1encmpcxc/content-images/jennifer-lawrence.png
upload: data/input/content-images/sphynx_cat.jpg to s3://sagemaker-studio-sr1encmpcxc/content-images/sphynx_cat.jpg
upload: data/input/content-images/lavaux.jpeg to s3://sagemaker-studio-sr1encmpcxc/content-images/lavaux.jpeg
upload: data/input/content-images/monalisa.jpg to s3://sagemaker-studio-sr1encmpcxc/content-images/monalisa.jpg
upload: data/input/content-images/debarcadere.jpeg to s3://sagemaker-studio-sr1encmpcxc/content-images/debarcadere.jpeg


In [13]:
%%capture
!aws s3 cp  ./data/input/style-images  {STYLE_IMAGES_BUCKET}  --recursive

## Run estimators

#### Setup and Fit estimators

In [76]:
#estimator_names = []

num_steps = 300
num_iter = 2
style_weight = 1e6
content_weight = 1e1

content_image = 'sphynx_cat.jpg'

style_images = [
    'Joan_Miro_39.jpg',
    'Pablo_Picasso_416.jpg',
    'Paul_Klee_23.jpg',
    'Paul_Klee_58.jpg',
    'Paul_Klee_95.jpg',
    'Pierre-Auguste_Renoir_23.jpg',
    'Vincent_van_Gogh_575.jpg',
    'Pierre-Auguste_Renoir_52.jpg',
    'Pierre-Auguste_Renoir_250.jpg',
    'Andy_Warhol_66.jpg',
    'Pablo_Picasso_130.jpg',
    'Edvard_Munch_3.jpg',
    'Georges_Seurat_23.jpg',
    'Vincent_van_Gogh_313.jpg',
    'Vincent_van_Gogh_69.jpg',
    'Vincent_van_Gogh_872.jpg',
]

for style_img in style_images:
    
    hyperparameters = {
        "content_image":content_image ,
        "style_image": style_img,
        "num_steps": num_steps,
        "num_iter": num_iter,
        "style_weight": style_weight,
        "content_weight": content_weight
    }
    hyperparams_keys= list(hyperparameters.keys())
    
    estimator = PyTorch(
        entry_point="style_transfer_pytorch.py",
        base_job_name='pytorch-style-transfer22',
        role=role,
        framework_version="1.8.0",
        instance_count=1,
        instance_type="ml.m5.large",
        py_version='py3',
        hyperparameters=hyperparameters,
        output_path=OUTPUT_BUCKET # SM_OUTPUT_DATA_DIR
    )


    channels = {
        "CONTENT_DATA": CONTENT_IMAGES_BUCKET, #'SM_CHANNEL_CONTENT_DATA'
        "STYLE_DATA": STYLE_IMAGES_BUCKET, #SM_CHANNEL_STYLE_DATA
        "SOMETHING-else": "s3://sagemaker-studio-sr1encmpcxc/", #SM_CHANNEL_SOMETHING-ELSE
    }
    
    estimator.fit(channels, wait=False)
      
    estimator_names.append(estimator._current_job_name)
    
estimator_names

INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating training-job with name: pytorch-style-transfer22-2023-04-15-12-42-10-103


['pytorch-style-transfer22-2023-04-15-11-09-01-111',
 'pytorch-style-transfer22-2023-04-15-11-09-02-400',
 'pytorch-style-transfer22-2023-04-15-11-09-03-496',
 'pytorch-style-transfer22-2023-04-15-11-09-04-198',
 'pytorch-style-transfer22-2023-04-15-11-09-05-115',
 'pytorch-style-transfer22-2023-04-15-11-09-06-670',
 'pytorch-style-transfer22-2023-04-15-11-09-07-688',
 'pytorch-style-transfer22-2023-04-15-11-09-08-256',
 'pytorch-style-transfer22-2023-04-15-11-09-09-503',
 'pytorch-style-transfer22-2023-04-15-11-38-34-217',
 'pytorch-style-transfer22-2023-04-15-12-05-36-695',
 'pytorch-style-transfer22-2023-04-15-12-05-37-456',
 'pytorch-style-transfer22-2023-04-15-12-05-43-908',
 'pytorch-style-transfer22-2023-04-15-12-05-44-512',
 'pytorch-style-transfer22-2023-04-15-12-05-45-086',
 'pytorch-style-transfer22-2023-04-15-12-05-46-107',
 'pytorch-style-transfer22-2023-04-15-12-05-46-909',
 'pytorch-style-transfer22-2023-04-15-12-42-10-103']

## Save estimators results

In [46]:
def save_estimator_hyperparams(estimator):
    estimator_name = eval(estimator.hyperparameters()['sagemaker_job_name'])
    estimator_file_path = f"{METADATA_LOCAL_PATH}/{estimator_name}.json"
    with open(estimator_file_path, "w") as f:
        json.dump(estimator.hyperparameters(), f)

        
def download_output_images(estimator) -> str:
    estimator_name = eval(estimator.hyperparameters()['sagemaker_job_name'])
    
    image_source_path = f'{OUTPUT_BUCKET}{estimator_name}/output/output.tar.gz'
    tar_image_dest_path = f'{OUTPUT_DOWNLOADS_PATH}/{estimator_name}-output.tar.gz'
    image_dest_path = f'{OUTPUT_DOWNLOADS_PATH}/{estimator_name}'
    
    ! aws s3 cp {image_source_path}  {tar_image_dest_path}
    ! mkdir -p {image_dest_path}
    ! tar -xzf {tar_image_dest_path} -C {image_dest_path} --no-same-owner
    ! rm {tar_image_dest_path}


def save_experiment(estimator, hyperparams_keys):
    hyperparams = estimator.hyperparameters()
    
    content_img_name = eval(hyperparams['content_image'])
    style_img_name = eval(hyperparams['style_image'])
    estimator_name = eval(hyperparams['sagemaker_job_name'])
    output_images_dir = f'{OUTPUT_DOWNLOADS_PATH}/{estimator_name}'

    # Define input paths for the images
    content_image_path = f'{INPUT_CONTENT_PATH}/{content_img_name}'
    style_image_path = f'{INPUT_STYLE_PATH}/{style_img_name}'
    output_image_path = final_image_path(output_images_dir, eval(hyperparams['num_iter']))
    
    # Define output path for the results summary image
    output_experiment_path = f'{OUTPUT_RESULTS_PATH}/{estimator_name}.jpg'
    
    # Load images
    images = [
        ('content_image', Image.open(content_image_path)),
        ('style_image', Image.open(style_image_path)),
        ('output_image', Image.open(output_image_path))
    ]

    # Plot images
    fig, axs = plt.subplots(1, 3,  figsize=(15, 5))
    for (img_type, img_data), ax in zip(images, axs):
        ax.imshow(img_data)
        ax.axis('off')
        ax.set_title(img_type)
     
    title = " || ".join([f"{key}: {hyperparams.get(key)}" for key in hyperparams_keys])
    plt.suptitle(title)
    fig.tight_layout(rect=[0, 0.03, 1, 0.95])
    fig.savefig(output_experiment_path, dpi=100, bbox_inches='tight')
        
        
def final_image_path(output_images_dir: str, num_iter: int) -> str:
    return f'{output_images_dir}/result-iter-{num_iter - 1}.jpg'

    
# # Uncomment to load an existing estimator and test
# estimator = Estimator.attach("pytorch-style-transfer22-2023-04-11-09-06-19-859") 
# download_output_images(estimator)
# save_estimator_hyperparams(estimator)
# save_experiment(estimator,)

In [88]:
hyperparams_keys = ['content_image', 'style_image', 'num_steps', 'num_iter', 'style_weight', 'content_weight']
failed_jobs = set(['pytorch-style-transfer22-2023-04-15-11-09-02-400', 'pytorch-style-transfer22-2023-04-15-12-42-10-103'])

incomplete_jobs = set(estimator_names) - failed_jobs
print('Waiting for incomplete jobs:',incomplete_jobs)

completed_jobs = set()

sm = boto3.Session().client('sagemaker')

while incomplete_jobs:
    
    for job_name in incomplete_jobs:
        
        status = sm.describe_training_job(TrainingJobName=job_name)['TrainingJobStatus']
        
        if status == 'Completed':
            
            print(f'Completed job: {job_name}')
            completed_jobs.add(job_name)
            output_results_img_path = f'{OUTPUT_RESULTS_PATH}/{job_name}.jpg'
            
            if not os.path.isfile(output_results_img_path): 
                
                estimator = Estimator.attach(job_name) 
                save_estimator_hyperparams(estimator)
                download_output_images(estimator)
                save_experiment(estimator, hyperparams_keys)
                
            
    if completed_jobs:       
        incomplete_jobs = incomplete_jobs - completed_jobs
        print('Waiting for incomplete jobs:',incomplete_jobs)

    sleep(30)

Waiting for incomplete jobs: {'pytorch-style-transfer22-2023-04-15-11-09-04-198', 'pytorch-style-transfer22-2023-04-15-11-09-03-496', 'pytorch-style-transfer22-2023-04-15-12-05-45-086', 'pytorch-style-transfer22-2023-04-15-12-05-46-107', 'pytorch-style-transfer22-2023-04-15-11-38-34-217', 'pytorch-style-transfer22-2023-04-15-11-09-09-503', 'pytorch-style-transfer22-2023-04-15-11-09-07-688', 'pytorch-style-transfer22-2023-04-15-12-05-36-695', 'pytorch-style-transfer22-2023-04-15-12-05-44-512', 'pytorch-style-transfer22-2023-04-15-11-09-01-111', 'pytorch-style-transfer22-2023-04-15-11-09-08-256', 'pytorch-style-transfer22-2023-04-15-12-05-43-908', 'pytorch-style-transfer22-2023-04-15-11-09-05-115', 'pytorch-style-transfer22-2023-04-15-11-09-06-670', 'pytorch-style-transfer22-2023-04-15-12-05-37-456', 'pytorch-style-transfer22-2023-04-15-12-05-46-909'}
Completed job: pytorch-style-transfer22-2023-04-15-11-09-04-198
Completed job: pytorch-style-transfer22-2023-04-15-11-09-03-496
Completed 

In [76]:
sm.describe_training_job(TrainingJobName="pytorch-style-transfer22-2023-04-11-09-06-19-859")

{'TrainingJobName': 'pytorch-style-transfer22-2023-04-11-09-06-19-859',
 'TrainingJobArn': 'arn:aws:sagemaker:us-east-1:798714931926:training-job/pytorch-style-transfer22-2023-04-11-09-06-19-859',
 'ModelArtifacts': {'S3ModelArtifacts': 's3://sagemaker-studio-sr1encmpcxc/output/pytorch-style-transfer22-2023-04-11-09-06-19-859/output/model.tar.gz'},
 'TrainingJobStatus': 'Completed',
 'SecondaryStatus': 'Completed',
 'HyperParameters': {'content_image': '"sphynx_cat.jpg"',
  'num_iter': '4',
  'num_steps': '300',
  'sagemaker_container_log_level': '20',
  'sagemaker_job_name': '"pytorch-style-transfer22-2023-04-11-09-06-19-859"',
  'sagemaker_program': '"style_transfer_pytorch.py"',
  'sagemaker_region': '"us-east-1"',
  'sagemaker_submit_directory': '"s3://sagemaker-studio-sr1encmpcxc/pytorch-style-transfer22-2023-04-11-09-06-19-859/source/sourcedir.tar.gz"',
  'style_image': '"Vassily_Kandinsky.jpeg"'},
 'AlgorithmSpecification': {'TrainingImage': '763104351884.dkr.ecr.us-east-1.amazo